In [1]:
import sys
import pickle
from scipy import signal
from scipy import stats
import numpy as np
from sklearn.model_selection import ShuffleSplit
from sklearn.metrics import cohen_kappa_score

import math
from collections import OrderedDict


import matplotlib.pyplot as plt

sys.path.append('D:\Diamond\code')
from csp_james_2 import *

sys.path.append('D:\Diamond\code')
from thesis_funcs_19_03 import *

import torch
import torch.nn as nn
import torch.nn.functional as nnF
import torch.optim as optim
from torch.autograd import Variable
from torch.optim import lr_scheduler

import csv
from random import randint
import random
import datetime
import scipy

In [74]:
save_root  = 'E:\\Diamond\\bci_iv\\MODELS\\fbcsp_mibif_cnn\\2a\\CURRENT\\NBPW\\'
config_root= 'E:\\Diamond\\bci_iv\\MODELS\\fbcsp_mibif_cnn\\2a\\configs\\'
raw_data_root = 'E:\\Diamond\\bci_iv\\DATA\\2a\\extract_raw\\'
feature_root = 'E:\\Diamond\\bci_iv\\MODELS\\fbcsp_mibif_cnn\\2a\\CURRENT\\gold_stand\\'


k_fold = 10


# initialize csp
m = 2# m is Nw in the paper "learning temporal information for brain-copmuter interface, Sakhavi et.al"
n_components = 2 * m  # pick some components
down_sample_step = 20 #Hilbert evelope
# select Ns pairs of csp filters
Ns = 4

CLASSES =[0,1]

C_OVR = [0,1,2,3]

balance_classes = 1


########################################################################################################################
                                            # DEFINE FILTER BANK
########################################################################################################################
#Filter Bank
FB = [[4., 8.], [8., 12.], [12., 16.], [16., 20.], [20., 24.], [24., 28.], [28., 32.], [32., 36.], [36., 40.]]
FB = np.array(FB)

#argumaents for Chebyl II filtering
# Nyquist frequency

# min. attenuation in stop band
gstop = 50
# max. attenuation in passband
gpass= 3 


EEG_PERIOD = [[2.5,6]]
FS = [250]


to_center_matrix = 0
calc_kl_d = 0
calc_mi_discrim  =0

In [3]:
# read _lambda values
with open(config_root +'_lambda_config.csv', mode = 'r') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter = ',')
    for row in csv_reader:
        _lambda_list_str = row[1:]
csv_file.close()        

#create _lambda value in list
_lambda_list = []
for i in _lambda_list_str:
    _lambda_list.append(float(i))

In [15]:
_lambda_list[0]

0.0

In [84]:
 #set load_idx to 0 if we want to create new and overwirite old T_IND files
k_file_root = None
fs = FS[0]
eeg_period = EEG_PERIOD[0]
COVS_AL_FD,  EEG_filt_FB_go, LABELS0_go, TRAIN_IDX, TEST_IDX, Train_idx = calc_k_covs_all_fold_current(filename_T, raw_data_root, 
                                                                                                            portion, k_file_root, 
                                                                                                            balance_classes, 
                                                                                                   FB,  gpass, gstop, fs, eeg_period, 
                                                                                                   k_fold, C_OVR, load_idx = 0, 
                                                                                                   cv_train_size = 0.7, 
                                                                                                 to_center_matrix = to_center_matrix, calc_covs_all_fd = 0)

D:\THINKPAD\Anaconda3\lib\site-packages\scipy\signal\_arraytools.py:45: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  b = a[a_slice]


In [83]:
np.shape(TRAIN_IDX)

(10, 189)

In [79]:
LABELS0_go1 = pickle.load(open(file_root_feature + '\\LABELS0_go.pickle', 'rb'))

In [33]:
portion = 1
portion_train = str(int(portion*100)) #mistake carried over - cv_train is actually the portion of training data used, and the directory name
cv_train_size = 0.5 #cv_train_size if the actually cv split


for subject in range(1, 2):
    filename_T = 'A0'+str(subject)+'T'
    file_root_feature =  feature_root + filename_T[:-1] + '\\4s\\' + 'pt_' + portion_train
    LABELS0_go = pickle.load(open(file_root_feature + '\\LABELS0_go.pickle', 'rb'))
    TRAIN_IDX = pickle.load(open(file_root_feature  + '\\TRAIN_IDX.pickle', 'rb'))
    TEST_IDX = pickle.load(open(file_root_feature + '\\TEST_IDX.pickle', 'rb'))  
    
    for fold in range (0, 1):
        #print (fold)

        #load in csp filters and mutual informtaion ranked indicies
        W_B = pickle.load(open( file_root + 'pt_' + portion_train
                                            + '\\W_B_fold_' + str(fold) + 
                                   '_c_ovr_' + str(c_ovr) + '_lambda_' + str(_lambda_list[0]) + 
                                   ".pickle", 'rb'))


        FB_FILTER_IND = pickle.load(open( file_root + 'pt_' + portion_train
                                            + '\\FB_FILTER_IND_fold_' + str(fold) + 
                                   '_c_ovr_' + str(c_ovr) + '_lambda_' + str(_lambda_list[0]) + 
                                   ".pickle", 'rb'))


        #find the selected csp filters indicies
        FB_FILTER_IND_slt = find_selected_csp_filters(Ns, m, FB_FILTER_IND)

        #construct selected csp filters, W_B_slt has shape (2*Ns, num_chls), (8,22) for example
        W_B_slt = W_B[FB_FILTER_IND_slt[:,0], :, FB_FILTER_IND_slt[:,1]]
        

        
        train_idx0 = TRAIN_IDX[fold]
        test_idx0 = TEST_IDX[fold]

        trn_ind_map = np.arange(0,len(train_idx0))
        tst_ind_map = np.arange(0,len(test_idx0))

        random.shuffle(trn_ind_map)
        random.shuffle(tst_ind_map)

        y_train = LABELS0_go[train_idx0[trn_ind_map]]
        y_test = LABELS0_go[test_idx0[tst_ind_map]]

        EEG_
        
        
        
        

        
        
        X_train0 = pickle.load(open(file_root_feature + '\\Z_all_classes_train_fold_' + str(fold) + 
                                    '_lambda_' + str(_lambda_list[0]) + ".pickle", 'rb'))

        X_test0 = pickle.load(open( file_root_feature + '\\Z_all_classes_test_fold_' + str(fold) + 
                                   '_lambda_' + str(_lambda_list[0]) + ".pickle", 'rb'))


        X_train0 = X_train0[trn_ind_map]
        X_test0 = X_test0[tst_ind_map]
        
        X_train = np.reshape(X_train0, (len(X_train0), -1))
        X_test = np.reshape(X_test0, (len(X_test0), -1))
        X_train = np.transpose(X_train)
        X_test = np.transpose(X_test)
        
        P_w = [0.25, 0.25, 0.25, 0.25]
        CLASSES = [0,1,2,3]
        
        PRED = []
        PPP = []

        for tr in range (0, 1):
            pred, P_w_x = pred_NBPW(X_test[:, tr:tr+1], X_train, CLASSES, y_train, P_w)
            PRED.append(pred)
            PPP.append(P_w_x) 

In [56]:
np.shape(np.sum(X_train0 * X_train0, axis = -1))

(189, 32)

In [61]:
F_selected = np.sum(X_train0 * X_train0, axis = -1) / 44
LABELS_train = y_train
test_f = np.sum(X_test0 * X_test0, axis = -1) / 44 
test_f = test_f[0:1]
F_selected = np.transpose(F_selected)
test_f = np.transpose(test_f)

#P_x_w = np.array([1., 1. ])
P_x_w = np.ones(np.shape(CLASSES))

for j in range (0, int(np.shape(F_selected)[0])):
    print (j, P_x_w)

    OE = np.empty(len(CLASSES))
    P_xj_w = np.empty(len(CLASSES))


    for w in CLASSES:
        CLASS_idx_train = np.array(np.where(LABELS_train.squeeze() == w)).squeeze()

        #equa.19
        OEw = pred_OE(test_f, F_selected, j, CLASS_idx_train)
        OE[w] = OEw

    P_xj_w=  1./len(CLASS_idx_train) * OE

    #equa.18
    P_x_w = P_x_w * P_xj_w

#equa.17
P_x = sum(P_x_w * P_w)

P_w_x = P_x_w * P_w / P_x

pred = np.argmax(P_w_x)


0 [1. 1. 1. 1.]
1 [0.00770057 0.14850726 0.06081744 0.11021252]
2 [0.0006902  0.02475718 0.00645558 0.0131924 ]
3 [0.00013152 0.00451105 0.0001445  0.00032084]
4 [2.22707634e-05 1.03407684e-04 1.79539234e-05 3.88271955e-05]
5 [3.14344387e-06 2.78135819e-05 1.76225032e-06 4.75693208e-06]
6 [4.25302949e-07 3.08811597e-06 2.06301896e-07 5.41843957e-07]
7 [5.34809877e-08 2.10022784e-07 1.08466846e-08 3.61413496e-08]
8 [5.25515315e-09 3.54077705e-08 1.73216189e-09 2.94001958e-09]
9 [8.45330862e-10 2.29424346e-09 1.04051659e-10 1.76702855e-10]
10 [1.66854327e-10 7.67288897e-10 2.45683708e-11 3.37285548e-11]
11 [5.74851869e-12 7.56509417e-82 3.11121841e-12 2.87905281e-12]
12 [4.53515973e-13 1.20074666e-82 3.85541421e-13 2.78053498e-13]
13 [5.12991491e-14 1.64537991e-83 3.79102261e-14 1.83338178e-14]
14 [8.97897198e-15 3.84138287e-84 4.84417438e-15 2.27536234e-15]
15 [1.90423489e-15 2.55578158e-85 4.26939297e-16 2.69016033e-17]
16 [5.18300367e-17 1.07446290e-86 1.60164800e-17 3.35726148e-19]
1

In [60]:
np.shape(test_f)

(1, 32)

In [39]:
np.ones(len(CLASSES)) * np.array([7,7,8,9])

array([7., 7., 8., 9.])

In [28]:
np.where(y_train == 0)[0]

array([  5,   8,  21,  22,  25,  28,  32,  39,  43,  45,  49,  60,  61,
        63,  67,  71,  73,  78,  80,  85,  91,  93,  95,  99, 100, 106,
       109, 110, 112, 118, 119, 120, 122, 124, 128, 130, 132, 139, 143,
       147, 157, 161, 164, 173, 176, 177, 185, 187], dtype=int64)

In [65]:
 P_w = [0.25, 0.25, 0.25, 0.25]

pred, P_w_x = pred_NBPW(np.transpose(X_test[0:1]), np.transpose(X_train), [0,1,2,3], y_train, P_w)

ValueError: operands could not be broadcast together with shapes (2,) (4,) 

In [63]:
np.shape(np.transpose(X_test))

(1408, 84)

In [62]:
np.shape( np.transpose(X_train))

(1408, 189)

In [66]:
len([0,1,2,3])

4

In [68]:
np.where(y_train.squeeze() ==3)

(array([  8,   9,  11,  13,  18,  19,  25,  34,  37,  41,  43,  51,  53,
         57,  60,  62,  68,  75,  76,  77,  85,  88,  89,  92,  95,  96,
         98, 102, 107, 111, 116, 120, 127, 129, 138, 144, 150, 151, 156,
        157, 158, 162, 171, 178, 179, 184], dtype=int64),)

In [69]:
np.ones(np.shape([0,1,2,3]))

array([1., 1., 1., 1.])

In [ ]:
portion = 0.1
portion_train = str(int(portion*100)) #mistake carried over - cv_train is actually the portion of training data used, and the directory name
cv_train_size = 0.5 #cv_train_size if the actually cv split

for subject in range(6, 7):
    print (subject)
    filename_T = 'A0'+str(subject)+'T'
    filename_E = 'A0'+str(subject)+'E'
    #ilename_save = filename
    #rint (filename_save)
    
    file_root  = save_root + 'A0'+str(subject) + '\\4s\\'
    k_file_root = file_root + 'pt_'+ portion_train + '\\'
    
    eeg_period = EEG_PERIOD[0]
    fs = FS[0]
    
    #set load_idx to 0 if we want to create new and overwirite old T_IND files
    COVS_AL_FD,  EEG_filt_FB_go, LABELS0_go, TRAIN_IDX, TEST_IDX, Train_idx = calc_k_covs_all_fold_current(filename_T, raw_data_root, 
                                                                                                            portion, k_file_root, 
                                                                                                            balance_classes, 
                                                                                                   FB,  gpass, gstop, fs, eeg_period, 
                                                                                                   k_fold, C_OVR, load_idx = 0, 
                                                                                                   cv_train_size = cv_train_size, 
                                                                                                 to_center_matrix = to_center_matrix)
    
    if to_save == 1:
        pickle.dump(TRAIN_IDX , open(file_root +'pt_'+ portion_train + '\\TRAIN_IDX' + ".pickle", "wb" ) )

        pickle.dump(TEST_IDX , open(file_root +'pt_'+ portion_train + '\\TEST_IDX' + ".pickle", "wb" ) )

        pickle.dump(LABELS0_go , open(file_root +'pt_'+ portion_train + '\\LABELS0_go' + ".pickle", "wb" ) )
        
        
        
     ########################################################################################################################

    for fold in range (0, k_fold):
        #print ('fold', fold)
        for _lambda in _lambda_list[0:1]:
            for c_ovr in C_OVR:
                #print (c_ovr)


                #load in csp filters and mutual informtaion ranked indicies
                W_B = pickle.load(open( file_root + 'pt_' + portion_train
                                                    + '\\W_B_fold_' + str(fold) + 
                                           '_c_ovr_' + str(c_ovr) + '_lambda_' + str(_lambda) + 
                                           ".pickle", 'rb'))


                FB_FILTER_IND = pickle.load(open( file_root + 'pt_' + portion_train
                                                    + '\\FB_FILTER_IND_fold_' + str(fold) + 
                                           '_c_ovr_' + str(c_ovr) + '_lambda_' + str(_lambda) + 
                                           ".pickle", 'rb'))




                #find the selected csp filters indicies
                FB_FILTER_IND_slt = find_selected_csp_filters(Ns, m, FB_FILTER_IND)

                #construct selected csp filters, W_B_slt has shape (2*Ns, num_chls), (8,22) for example
                W_B_slt = W_B[FB_FILTER_IND_slt[:,0], :, FB_FILTER_IND_slt[:,1]]

